# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
# Importando bibliotecas
import pandas as pd
import patsy as pt
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [4]:
# Reduzindo o número de colunas analizadas
# Retirando as colunas aleatórias de identificação (Unnamed: 0, data_ref, id_cliente)

analise = df.iloc[:, 3:]
analise.head(1)

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.60274,1.0,8060.34


1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.
    

In [5]:
#Selecionar a frequência de cada resposta qualitativa:
# Selecionar as colunas desejadas
colunas = ['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']
contagem = analise[colunas].melt()

# Contar os valores únicos em todas as colunas
contagem.value_counts().sort_index(level=0)

variable          value              
educacao          Primário                 165
                  Pós graduação             26
                  Secundário              8895
                  Superior completo       5335
                  Superior incompleto      579
estado_civil      Casado                 10534
                  Separado                 879
                  Solteiro                1798
                  União                   1078
                  Viúvo                    711
posse_de_imovel   False                   4857
                  True                   10143
posse_de_veiculo  False                   9140
                  True                    5860
sexo              F                      10119
                  M                       4881
tipo_renda        Assalariado             7633
                  Bolsista                   9
                  Empresário              3508
                  Pensionista             2582
                  Serv

In [7]:
# Variáveis qualitativas como dummies:
# As variáveis qualitativas são sexo, educacao, posso_de_veiculo, posse_de_imoveis, tipo_renda, estado_civil e tipo_residencia
# Considerando as variáveis mais frequentes, observando a contagem anterior

y, x = pt.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + C(educacao, Treatment (2)) + C(posse_de_imovel, Treatment(1)) + tipo_renda + estado_civil + C(tipo_residencia, Treatment(1))', data = analise)
print("Análise: \n", sm.OLS(y, x).fit().summary())

Análise: 
                             OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     144.7
Date:                Thu, 13 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:08:49   Log-Likelihood:                -18100.
No. Observations:               15000   AIC:                         3.624e+04
Df Residuals:                   14979   BIC:                         3.640e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------

Os parêmtros analisados estão coerentes com os valores esperados. 
* O R² é 0.162, próximo a 0, não sendo, portanto, um bom modelo de análise.
* Em relação ao valor de confiabilidade, as variáveis posse_de_veiculo = True, estado_civil = Viúvo, tipo_renda = Bolsista, educacao = Primário, educacao = Pós graduação e tipo_residencia = Governamental são as únicas com valores P superiores, ou próximo, a 5%, indicando que são estatisticamente não significativas no modelo.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

In [8]:
# As variáveis de educação são os indicadores com maior valor de P>|t|, assim:
y, x = pt.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + C(posse_de_imovel, Treatment(1)) + tipo_renda + estado_civil + C(tipo_residencia, Treatment(1))', data = analise)
print("Análise: \n", sm.OLS(y, x).fit().summary())

Análise: 
                             OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     176.1
Date:                Thu, 13 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:19:59   Log-Likelihood:                -18132.
No. Observations:               15000   AIC:                         3.630e+04
Df Residuals:                   14983   BIC:                         3.643e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------

* Houve uma pequena redução dp R², de 0,158. 
* Em relação aos valores de confiabilidade houveram pequenas alterações dos valores, sendo em sua maioria reduções.  

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 

In [9]:
# Retirando tipo de residência que é o maior valor de R², 0,276, após a questão anterior 
y, x = pt.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + C(posse_de_imovel, Treatment(1)) + tipo_renda + estado_civil', data = analise)
print("Análise: \n", sm.OLS(y, x).fit().summary())

Análise: 
                             OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     252.8
Date:                Thu, 13 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:21:28   Log-Likelihood:                -18148.
No. Observations:               15000   AIC:                         3.632e+04
Df Residuals:                   14988   BIC:                         3.641e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------

In [10]:
# Retirando estado_civil que é o maior valor de R², 0,251, após o passo anterior 
y, x = pt.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + C(posse_de_imovel, Treatment(1)) + tipo_renda', data = analise)
print("Análise: \n", sm.OLS(y, x).fit().summary())

Análise: 
                             OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     394.3
Date:                Thu, 13 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:22:35   Log-Likelihood:                -18157.
No. Observations:               15000   AIC:                         3.633e+04
Df Residuals:                   14992   BIC:                         3.639e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------

In [11]:
# Retirando posse_de_veiculo que é o maior valor de R², 0,177, após o passo anterior 
y, x = pt.dmatrices('np.log(renda) ~ sexo + C(posse_de_imovel, Treatment(1)) + tipo_renda', data = analise)
print("Análise: \n", sm.OLS(y, x).fit().summary())

Análise: 
                             OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     459.7
Date:                Thu, 13 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:23:49   Log-Likelihood:                -18158.
No. Observations:               15000   AIC:                         3.633e+04
Df Residuals:                   14993   BIC:                         3.638e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------

In [12]:
# Retirando tipo_renda que é o maior valor de R², 0,097, após o passo anterior 
y, x = pt.dmatrices('np.log(renda) ~ sexo + C(posse_de_imovel, Treatment(1))', data = analise)
print("Análise: \n", sm.OLS(y, x).fit().summary())

Análise: 
                             OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     1248.
Date:                Thu, 13 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:25:03   Log-Likelihood:                -18270.
No. Observations:               15000   AIC:                         3.655e+04
Df Residuals:                   14997   BIC:                         3.657e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------

* Após a exclusão das variáveis houve uma pequena redução no valor do R², aproximandamente, 0,019, quando comparado ao modelo inicial. 
* Como os valores de P>|t| são iguais e o R² é menor, inidicando não haver erro no modelo, portanto  esse modelo parace melhor. 